In [1]:
import altair as alt
import pandas as pd
import requests
import codecs
from requests.exceptions import HTTPError
import openpyxl

In [2]:
#add export?format=xlsx after id
url = "https://docs.google.com/spreadsheets/d/1pUDOO5rROKI20sP5p5_2gVCMvsOoBoFzYUNtz2fZYqA/export?format=xlsx"

In [3]:

try:
  #Send get request
  response = requests.get(url)
  # If the response was successful, no Exception will be raised
  response.raise_for_status()
except HTTPError as http_err:
  print(f'HTTP error occurred: {http_err}')
except Exception as err:
  print(f'Other error occurred: {err}')
else:
  print('Request successfully!')

Request successfully!


In [4]:
excelFileName = "excel.xlsx"

writeSuccessful = True
try:
  with codecs.open(excelFileName, 'wb') as f:
    f.write(response.content)
except Exception as err:
  print(f"Error occured when writing file: {err}")
  writeSuccessful = False
else:
  print("Excel file written successfully")


Excel file written successfully


In [5]:
#Get the sheet names in excel. 
sheetNames = openpyxl.load_workbook(excelFileName, read_only=True).sheetnames
sheetNamesDict = {s:'' for s in sheetNames}
print(f"sheetNamesDict = {sheetNamesDict}")

sheetNamesDict = {'ИТБ': '', 'ФЭТ': '', 'ФМ': ''}


In [6]:
#altair does not work with russian characters. Define sheet names in english
sheetNamesDict = {'ИТБ': 'ITMB', 'ФЭТ': 'FET', 'ФМ': 'FM'}

In [7]:
if writeSuccessful:
  dataSource = excelFileName
else:
  dataSource = response.content

#Read pandas dataframes from Excel file that was saved
dfs = []
for sheetNameRu in sheetNamesDict:
  df = pd.read_excel(dataSource, header=1, sheet_name=sheetNameRu)
  sheetNameEn = sheetNamesDict[sheetNameRu]
  df['Program'] = sheetNameEn
  dfs.append(df)

In [8]:
#Concatenate the dataframes from all sheets
df = pd.concat(dfs, ignore_index=True)

#Drop the columns which have all nan values
df = df.dropna(axis='columns', how = 'all')

In [9]:
columnNamesDict = {col : '' for col in list(df.columns)} 
print(f"columnNamesDict = {columnNamesDict}")

columnNamesDict = {'№': '', 'ФИО': '', 'сумма баллов': '', 'Мат.': '', 'Русс.': '', 'ИКТ': '', 'Бюджет/договор': '', 'Program': '', 'Ин.яз': ''}


In [10]:
#Remove the columns that you don't need and provide the english translation for the columns left
#columnNamesDict = {'№': '', 'ФИО': '', 'сумма баллов': '', 'Мат.': '', 'Русс.': '', 'ИКТ': '', 'Бюджет/договор': '', 'Program': '', 'Ин.яз': ''}
columnNamesDict = {'ФИО': 'Name', 'сумма баллов': 'Total Score', 'Мат.': 'Math', 'Русс.': 'Russian', 'ИКТ': 'IT', 'Program': 'Program', 'Ин.яз': 'Foreign language'}

In [11]:
#Filter only the needed columns
df = df[columnNamesDict.keys()]

#Replace the russian column names with the english stored in the dictionary
columnNamesEn = [columnNamesDict[col_ru] for col_ru in df.columns]
df.columns = columnNamesEn

In [12]:
df.head()

,Name,Total Score,Math,Russian,IT,Program,Foreign language
0,Абрамов Петр Александрович,243,75.0,98.0,70.0,ITMB,NaN
1,Амерханова Наргиза Артуровна,286,96.0,89.0,96.0,ITMB,NaN
2,Баранов Алексей Алексеевич,235,45.0,96.0,94.0,ITMB,NaN
3,Баубеков Мирас Булатулы,217,70.0,81.0,66.0,ITMB,NaN
4,Богун Владислав Олегович,242,78.0,85.0,79.0,ITMB,NaN


In [15]:

alt.Chart(df).mark_circle(size=100).encode(
    x='Math',
    y='Foreign language',
    color='Program',
    tooltip=['Name', 'Total Score']
).interactive()

alt.Chart(...)